In [ ]:
!pip install datasets
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 18.4 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 15.0 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

dataset = load_dataset("Skylion007/openwebtext", cache_dir="/content/drive/MyDrive/Colab\ Notebooks/openwebtext")
dataset

In [ ]:
from tqdm import tqdm
import numpy as np
from datasets import load_dataset
import tiktoken
import os

num_proc = 16 # Recommended as half the number of cpu cores by Andrej

enc = tiktoken.get_encoding("gpt2")

In [ ]:
# num_proc is part of huggingface dataset loading multiprocessing using multiple cpu cores
dataset = load_dataset("openwebtext", cache_dir="/content/drive/MyDrive/Colab\ Notebooks/openwebtext", num_proc = num_proc)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for openwebtext contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/openwebtext
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Loading dataset shards:   0%|          | 0/82 [00:00<?, ?it/s]

In [ ]:
split_dataset = dataset["train"].train_test_split(test_size=0.0005, seed=2357, shuffle=True)
split_dataset["val"] = split_dataset.pop('test')
split_dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 8009762
    })
    val: Dataset({
        features: ['text'],
        num_rows: 4007
    })
})

In [ ]:
# Function to tokenize the dataset

def process(example):
  ids = enc.encode_ordinary(example['text'])
  ids.append(enc.eot_token)
  out = {'ids': ids, 'len': len(ids)}
  return out

# tokenizing the dataset using huggingface .map() function
tokenized = split_dataset.map(
    process,
    remove_columns=['text'],
    desc="tokenizing the splits",
    num_proc=num_proc
)

tokenizing the splits (num_proc=16):   0%|          | 0/8009762 [00:00<?, ? examples/s]

TimeoutError: 

In [ ]:
output_dir = '/content/drive/MyDrive/Colab\ Notebooks/transfomers'

# concatenating all the ids in each dataset into one large file we can use for training
for split, dset in tokenized.items():
    arr_len = np.sum(dset['len'], dtype=uint64)
    filename = os.path.join(output_dir, f'{split}.bin')
    dtype = np.uint16 # 2**16 because its greater than largest token_id value
    arr = np.memmap(filename, dtype=dtype, mode='w+', shape=(arr_len,))
    total_batches = 1024

    idx = 0
    for batch_idx in tqdm(range(total_batches), desc=f'writing {filename}'):
        # Batch together samples for faster write
        batch = dset.shard(num_shards=total_batches, index=batch_idx, contiguous=True).with_format('numpy')
        arr_batch = np.concatenate(batch['ids'])
        # Write the concatenated data into memmap
        arr[idx:idx+len(arr_batch)] = arr_batch
        idx += len(arr_batch)
    arr.flush()

# To read the bin files later, e.g. with numpy
# m = np.memmap('train.bin'. dtype=np.uint16, mode='r')

NameError: name 'tokenized' is not defined

In [ ]:
from torch.utils.data import Dataset
import numpy as np
import torch

class MemmapDataset(Dataset):
    def __init__(self, data_dir, seq_len):
        self.dataset = np.memmap(os.path.join(data_dir, 'train.bin'), dtype=np.uint16, mode='r')
        self.seq_len = seq_len
        self.total_length = len(self.dataset) - seq_len

    def __len__(self):
        return self.total_length

    def __getitem__(self, idx):
        input_sequence = self.dataset[idx:idx+self.seq_len]
        target_sequence = self.dataset[idx+1:idx+self.seq_len+1]
        return torch.tensor(input_sequence, dtype=torch.long), torch.tensor(target_sequence, dtype=torch.long)

In [ ]:
from torch.utils.data import DataLoader

output_dir = '/content/drive/MyDrive/Colab\ Notebooks/transfomers'

dataset = MemmapDataset(output_dir, seq_len=512)

train_dataloader = DataLoader(dataset, batch_size=64, shuffle=True, num_workers=4)
val_dataloader = DataLoader(dataset, batch_size=64, shuffle=False, num_workers=4)

In [ ]:
# Fetch a batch from the training DataLoader
sample_batch = next(iter(train_loader))

# Check and print the types of the elements in the batch
input_sequences, target_sequences = sample_batch
print(input_sequences.shape)
print(target_sequences.shape)
print(f"Input type: {type(input_sequences)}")
print(f"Target type: {type(target_sequences)}")
print(input_sequences)
print(target_sequences)

In [ ]:
print(dataset['train'][1])
print(len(dataset['train']))
print(len(dataset['train'][1]['text']))

{'text': 'Former secretary of state Hillary Clinton meets voters at a campaign rally in St. Louis on Saturday. (Melina Mara/The Washington Post)\n\nDemocratic front-runner Hillary Clinton was ahead by a slim margin in Missouri on Wednesday, but the race remained in limbo pending word on whether rival Sen. Bernie Sanders of Vermont would seek a recount.\n\nThe delay postponed a definitive answer to whether Clinton had made a clean sweep of five big primaries on Tuesday night. Even if she does not prevail in Missouri, her other victories push her closer to the Democratic presidential nomination even as the considerably weakened Sanders vowed to press on with his insurgent campaign.\n\nClinton won big in Florida, North Carolina and Ohio, while claiming a narrower victory in Illinois. In Missouri, with 100 percent of precincts reporting, Clinton was ahead 310,602 votes to 309,071. With a difference of less than 1 percent, state officials held off calling the race. A recount is not automati

In [ ]:
print(enc.encode("Hello world!"))
print(enc.decode([50000]))

[15496, 995, 0]
 grids


In [ ]:
train = enc.encode(dataset['train'][9]['text'])
print(train)
print(max(train))
print(len(train))

[1135, 447, 247, 303, 1464, 17915, 29410, 544, 355, 262, 1363, 286, 1036, 271, 306, 4065, 10165, 11, 7650, 1461, 2647, 290, 314, 7336, 32, 13, 887, 340, 3073, 588, 612, 447, 247, 82, 257, 3957, 2183, 3715, 1165, 13, 198, 198, 2484, 2840, 588, 262, 370, 3532, 2144, 16683, 290, 30015, 8562, 5934, 5427, 761, 645, 9793, 13, 887, 3892, 706, 938, 1285, 447, 247, 82, 22158, 45483, 402, 4694, 2058, 428, 7702, 20059, 30013, 422, 17068, 23049, 8495, 286, 10710, 13, 198, 198, 1026, 468, 645, 5852, 11, 340, 447, 247, 82, 13055, 287, 257, 21002, 15723, 2635, 11, 290, 340, 447, 247, 82, 262, 2183, 7548, 286, 281, 4771, 12, 36673, 2823, 286, 11446, 615, 270, 13, 198, 198, 464, 27098, 318, 17068, 8124, 16528, 11, 257, 6073, 12, 1941, 12, 727, 11949, 508, 3160, 655, 2354, 29679, 13, 564, 250, 33, 7938, 326, 389, 3748, 393, 257, 1643, 564, 246, 732, 1447, 447, 247, 389, 644, 651, 502, 1016, 11, 447, 251, 339, 1139, 13, 564, 250, 23307, 867, 2183, 16715, 804, 262, 976, 26760, 13, 447, 251, 198, 198, 2002

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 64 # how many independent sequences will be processed in parallel
seq_len = 256 # the maximum context length for predictions
max_iters = 15000
eval_interval = 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 384 # same as d_model
n_head = 6
n_layer = 6
dropout = 0.2
EPOCHS = 20

In [ ]:
class Head(nn.Module):
    '''one head of self-attention'''
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(seq_len, seq_len)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # inputs: (batch, time-step, channels) == (batch_size, seq_len, d_model)
        # outputs: (batch, time-step, head size) == (batch_size, seq_len, d_k)
        B, T, C = x.shape
        k = self.key(x)   # (B, T, hs)
        q = self.query(x) # (B, T, hs)

        attention_scores = q @ k.transpose(-2, -1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T) WAIT HOLD UP THE BIGGEST MISTAKE WAS HERE
        attention_scores = attention_scores.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        attention_scores = F.softmax(attention_scores, dim=-1)
        attention_scores = self.dropout(attention_scores)

        v = self.value(x) # (B, T, hs)
        out = attention_scores @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.w_o = nn.Linear(head_size * num_heads, n_embd) # (d_k * h, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.w_o(out))
        return out

class FeedForward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd) # planning to make myself
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class GPTLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(seq_len, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        tok_emb = self.token_embedding_table(idx) # (B, T, C) == (Batchsize, seqlen, dmodel)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))
        x = tok_emb + pos_emb
        x = self.blocks(x) # (B, T, C)
        x = self.ln_f(x) # (B, T, C)
        logits = self.lm_head(x) # (B, T, vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -seq_len:]
            logits, loss = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

In [ ]:
model = GPTLanguageModel()
model = model.to(device)
print(device)

In [ ]:
print(sum(p.numel() for p in model.parameters() if p.requires_grad))

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [ ]:
for iter in range(max_iters):

    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: training loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

        # Log losses to W&B
        wandb.log({"Training Loss": losses['train'], "Validation Loss": losses['val'], "Iteration": iter})

    xb, yb = get_batch('train')

    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# Finish the W&B run
wandb.finish()

In [ ]:
for epoch in range(EPOCHS):
    for i, batch in enumerate(train_loader):
        input_sequences, target_sequences = batch
        input_sequences = torch.tensor(input_sequences, dtype=torch.long)
        target_sequences = torch.tensor(target_sequences, dtype=torch.long)
        input_sequences = input_sequences.to(device)
        target_sequences = target_sequences.to(device)

        with torch.no_grad():
            for val_batch in val_dataloader:
                val_input_sequences, val_target_sequences = val_batch

                logits, loss = model(val_input_sequences, val_target_sequences)
                val_loss = loss.item()

        if i % 8 == 0:
            print(f"Epoch: {epoch+1}/{EPOCHS}, Batch: {i+1}/{batch_size}")
        logits, loss = model(input_sequences, target_sequences)
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()


In [ ]:
@torch.no_grad() # Decorator that tells pytorch to not compute gradients during the operations performed in the decorated function
def estimate_loss():
    out={}
    model.eval()
    for split in ['train', 'val']: # list directly defined in a loop statement
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [ ]:
def load_checkpoint(model, optimizer, filename="gpt_checkpoint_1.53loss.pth"):
    checkpoint = torch.load(filename)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    print(f"Checkpoint loaded from {filename}")

load_checkpoint(model, optimizer)

In [ ]:
def save_checkpoint(model, optimizer, filename="gpt_checkpoint_1.53loss.pth"):
    checkpoint = {
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict()
    }
    torch.save(checkpoint, filename)
    print(f"Checkpoint saved to {filename}")

save_checkpoint(model, optimizer)

In [ ]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
print(decode(model.generate(context, max_new_tokens=500)[0].tolist()))